In [1]:
from brainlit.utils import upload, session
from pathlib import Path

## Uploading Brain Images from data in the Octree format.
This notebook demonstrates uploading the 2 lowest-resolution brain volumes, as well as a `.swc` segment file.
The upload destination could easily be set to a url of a cloud data server such as s3.

## 1) Define variables.
 - `source` and `source_segments` are the root directories of the octree-formatted data and swc files.
 - `p` is the prefix string. `file://` indicates a filepath, while `s3://` or `gc://` indicate URLs.
 - `dest` and `dest_segments` are the destinations for the uploads (in this case, filepaths).
 - `num_res` denotes the number of resolutions to upload.

In [2]:
source = str(Path().resolve().parents[2] / "data" / "data_octree")
p = "file://"
dest = str(Path().resolve().parents[2] / "data" / "upload")
dest_segments = str(Path().resolve().parents[2] / "data" / "upload_segments")
num_res = 2

## 2) Upload the image data.

In [3]:
upload.upload_volumes(source, p+dest, num_res)

Creating precomputed volume:   0%|          | 0/8 [00:00<?, ?it/s]
Finished mip 0, took 4.411967992782593 seconds
Creating precomputed volume: 100%|██████████| 8/8 [00:13<00:00,  1.72s/it]
Finished mip 1, took 13.773943901062012 seconds



In [4]:
upload.upload_segments(source, p+dest_segments, num_res)

Uploading: 100%|██████████| 1/1 [00:00<00:00, 152.60it/s]


## Visualize your data with NeuroglancerSession and napari

In [5]:
from cloudvolume import Bbox
from brainlit.utils.swc import graph_to_paths
import napari

In [6]:
ngl_sess = session.NeuroglancerSession(mip=1, url=p+dest, url_segments=p+dest_segments)
bounds = Bbox((0,0,0), (528,400,208))
img = ngl_sess.pull_bounds_img(bounds)
G_sub = ngl_sess.get_segments(2, bounds)
paths = graph_to_paths(G_sub)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 1006.07it/s]


In [7]:
with napari.gui_qt():
    viewer = napari.Viewer(ndisplay=3)
    viewer.add_image(img)
    viewer.add_shapes(data=paths, shape_type='path', edge_width=0.1, edge_color='blue', opacity=0.1)